In [1]:
dico_graphe = {}

In [2]:
def TrouveCommunaute(communautes, i):
    ''' Renvoie le numero de la communaute du noeud i'''
    
    return communautes[i]

In [3]:
def QuiDansCommunaute(communautes, C):
    ''' Renvoie une liste des noeuds appartenant la communauté C'''

    commu = []

    for n in range(len(communautes)):
        if communautes[n] == C:
            commu.append(n)
    
    return commu

In [4]:
def TrouveVoisins(dico, i):
    ''' Renvoie une liste des voisins du sommet i'''
    
    l_res = []
    v = dico[i]

    for j in v:
        l_res.append(j[0])

    return l_res

In [5]:
def AjoutDansCommunaute(communautes, i, C):
    ''' Ajoute i à la communauté C dans la liste communautés '''
    # communautes est une liste où chaque élément d'indice i est le numéro de la communauté de i

    communautes[i] = C
    return communautes

In [6]:
def MemeCommunaute(communautes, i, j):
    ''' Renvoie True si i et j sont dans la même communauté, False sinon'''

    if TrouveCommunaute(communautes, i) == TrouveCommunaute(communautes, j):
        return True
    return False

## Calcul du gain de modularité

In [7]:
def SommePoidsGraphe(dico):
    ''' Calcule la somme des poids du graphe (m) '''
    
    somme_totale = 0
    somme_intermediaire = 0

    for sommet, voisins in dico.items():
        for v in voisins:
            if sommet != v[0]:                      # si ce n'est pas une boucle => on la comptera deux fois
                somme_intermediaire += v[1]
            else:                                   # si c'est une boucle
                somme_totale += v[1]

    somme_totale += somme_intermediaire//2

    return somme_totale      


In [8]:
def SommeInside(dico, communautes, C:int):
    ''' Calcule la somme des poids des arêtes dans la communauté C'''
    
    noeuds_de_C = QuiDansCommunaute(communautes, C)
    somme = 0
    
    for noeud in noeuds_de_C:
        voisins = dico[noeud]
        for v in voisins:
            if TrouveCommunaute(communautes, v[0]) == C:
                somme += v[1]
    
    return somme//2

In [9]:
def SommeTotale(dico, communautes, C:int):
    ''' Calcule la somme des poids des arêtes sortant de C (de tous les sommets appartenant à C)'''
    
    noeuds_de_C = QuiDansCommunaute(communautes, C)
    somme = 0
    
    for noeud in noeuds_de_C:
        voisins = dico[noeud]
        for v in voisins:
            if (v[0] not in noeuds_de_C) or (v[0] > noeud):        # vérifier qu'on a pas déjà compté cette arête => attention jsp
                somme += v[1]
    
    return somme

In [10]:
def Ki(dico, i):
    ''' Calcule la somme des poids des arêtes sortant (entrant) de i'''
    
    return len(dico[i])

In [11]:
def GainModularite(dico, communautes, m, i:int, j:int):
    ''' Calcule le gain de modularité si on ajoute i à la communauté de j'''

    C = TrouveCommunaute(communautes, j)
    S_in = SommeInside(dico, communautes, C)
    k_i_in = KiInside(dico, communautes, i)
    S_tot = SommeTotale(dico, communautes, C)
    k_i = Ki(dico, i)

    Q = (((S_in + 2*k_i_in)/(2*m)) - ((S_tot + k_i)/(2*m))**2) - ((S_in/(2*m)) - (S_tot/(2*m))**2 - (k_i/(2*m))**2)

    return Q

## Algorithme de Louvain

In [12]:
def Louvain1(dico):
    ''' Première partie de l'algorithme de Louvain '''

    flag_modif = True
    m = SommePoidsGraphe(dico)
    # Initialisation de la liste communautes : chaque noeud est dans une communauté différente 
    communautes = [k for k in range(len(dico))]

    while flag_modif == True:
        flag_modif = False          # aucune modification n'a encore été faite pour cette itération
        for i in range(len(communautes)):        # pour chaque noeud i
            max_gain = -1
            max_j = -1
            voisins_de_i = TrouveVoisins(dico, i)
            for j in voisins_de_i:      # pour chaque voisin de i
                gain_mod = GainModularite(dico, communautes, m, i, j)      # calcul du gain de modularité
                if i == 2 :
                    print('i:', i, 'j:', j, 'gain:', gain_mod)
                if gain_mod > max_gain:          # /!\ gérer l'égalité
                    max_gain = gain_mod
                    max_j = j
            if i == 1 :
                print('max_gain :  ', max_gain, 'max_j : ', max_j)
            #if max_gain > 0:        # /!\ strict ou pas ?
            flag_modif = True
            print("j'ajoute ", i, " à la communauté de ", max_j)
            communautes = AjoutDansCommunaute(communautes, max_j, TrouveCommunaute(communautes, i))       # ajout de i dans la communauté de j
            print(max_j, " est maintenant dans la communauté numéro ", communautes[max_j], " et i dans la numéro ", communautes[i])
            print("communautes : ", communautes)
    return communautes

In [13]:
def Louvain12(dico):
    ''' Première partie de l'algorithme de Louvain '''

    flag_modif = True
    m = SommePoidsGraphe(dico)
    # Initialisation de la liste communautes : chaque noeud est dans une communauté différente 
    communautes = [k for k in range(len(dico))]

    while flag_modif == True:
        flag_modif = False          # aucune modification n'a encore été faite pour cette itération
        for i in range(len(communautes)):        # pour chaque noeud i
            max_gain = -1
            max_j = -1
            voisins_de_i = TrouveVoisins(dico, i)
            k_i_in = '' # fonction en question
            for j in voisins_de_i:      # pour chaque voisin de i
                gain_mod = GainModularite(dico, communautes, m, i, j)      # calcul du gain de modularité
                if i == 2 :
                    print('i:', i, 'j:', j, 'gain:', gain_mod)
                if gain_mod > max_gain:          # /!\ gérer l'égalité
                    max_gain = gain_mod
                    max_j = j
            if i == 1 :
                print('max_gain :  ', max_gain, 'max_j : ', max_j)
            #if max_gain > 0:        # /!\ strict ou pas ?
            flag_modif = True
            print("j'ajoute ", i, " à la communauté de ", max_j)
            communautes = AjoutDansCommunaute(communautes, max_j, TrouveCommunaute(communautes, i))       # ajout de i dans la communauté de j
            print(max_j, " est maintenant dans la communauté numéro ", communautes[max_j], " et i dans la numéro ", communautes[i])
            print("communautes : ", communautes)
    return communautes

## Nouvel essai !

In [14]:
def Calcul_degre(dico, i):
    """ Retourne le degré du noeud i"""
    
    return len(dico[i])

In [15]:
def Calcul_ki_inOLD(communautes, voisins_de_i, i, j):
    ''' Calcule la somme des poids des arêtes de i allant dans la communauté de j'''
    
    res = 0
    commu_de_j = communautes[j]

    for v in voisins_de_i:
        if communautes[v[0]] == commu_de_j:
            res += v[1]
    return res

#print(Calcul_ki_in([0, 1, 2, 1], [(1,1), (2,1), (3,1)], 0, 1))

In [16]:
def Calcul_ki(dico, i):
    ''' Calcule la somme des poids des arêtes du noeud i'''

    ki = 0

    for v in dico[i]:
        ki += v[1]

    return ki

In [17]:
def Calcul_S_tot(dico, noeuds_dans_commu, i):
    ''' Calcule la somme des poids des arêtes de la communauté testée '''
    # noeuds_dans_commu est une liste contenant les numéros des noeuds qui sont dans cette communauté
    # i est le noeud testé

    S_tot = 0

    for n in noeuds_dans_commu:
        if n != i:
            S_tot += Calcul_ki(dico, n)

    return S_tot

In [18]:
def Calcul_ki_in(dico, communautes, i, C):
    ''' Calcule la somme des poids des arêtes de i vers la communauté C'''

    ki_in = 0
    voisins = dico[i]

    for v in voisins:
        if communautes[v[0]] == C:
            ki_in += v[1]
    
    return ki_in


In [19]:
def renommeCommu(commu):
    set_commu = list(set(commu))

    li_correspondance = [i for i in range(len(set_commu))]

    for i in range(len(commu)):
        for j in range(len(set_commu)):
            if commu[i] == set_commu[j]:
                commu[i] = li_correspondance[j]
    return commu

In [49]:
def Louvain_P1(dico):
    
    flag_modif = True
    m = SommePoidsGraphe(dico)

    # Initialisation de la liste communautes : chaque noeud est dans une communauté différente 
    communautes = [k for k in range(len(dico))]

    while flag_modif == True:       # tant qu'on modifie encore les communautés
        flag_modif = False
        for noeud in range(len(communautes)):
            liste_voisins = TrouveVoisins(dico, noeud)
            max = -1
            max_v = -1
            
            liste_commu_voisins = []
            for vo in liste_voisins:            # pour chaque voisin
                liste_commu_voisins.append(communautes[vo])         # ajoute dans liste_commu_voisins le numéro de la communauté de ce voisin
                
            ki = Calcul_ki(dico, noeud)     # calcul du degré du noeud
            for v in liste_voisins:     # pour chaque voisin du noeud
                if v != noeud:
                    n_commu_v = QuiDansCommunaute(communautes, communautes[v])      # prendre la valeur [1]

                    # Calcul de ki_in
                    ki_in = Calcul_ki_in(dico, communautes, noeud, communautes[v])
                    # Calcul de S tot
                    S_tot = Calcul_S_tot(dico, n_commu_v, noeud)
                    # Calcul du gain
                    #gain = (1/(2*m))*((2*ki_in) - (S_tot * ki)/m)
                    gain = (2*ki_in) - (S_tot * ki)/m

#### tester positif
                    if gain > max:      # si c'est un meilleur gain
                        max = gain
                        max_v = v
            
            if (communautes[noeud] != communautes[max_v]) and (max > 0):        # si on doit mettre le noeud dans une communauté différente
                AjoutDansCommunaute(communautes, noeud, communautes[max_v])     # on le met dans cette communauté en question
                flag_modif = True       # on a modifié
                
            #print('communautes : ', communautes)
                       
            #print(noeud, max_v, max)

    return renommeCommu(communautes)
            

In [50]:
def Louvain_P2(dico_graphe, communautes):

    nouveau_graphe = {}
    nb_commu = len(set(communautes))

    for elt in range(nb_commu):
        li_sommet = []                          #récupère les sommets d'une communauté
        for sommet in range(len(communautes)):
            if communautes[sommet] == elt :
                li_sommet.append(sommet)

        poids_sommet_commu = 0                  #variable qui va contenir le poids du sommet (somme des aretes au sein de la commu)
        dico_arete_autre_commu = {}             #dico qui va contenir le nb d'aretes d'une commu vers chaque autres commu qui va correspondre au poids de l'arete reliant l'autre commu
        
        for sommet_commu in li_sommet :
            for arete in dico_graphe[sommet_commu]: #parcours les sommets de la commu en cours de traitement
                if arete[0] in li_sommet :       #parcours les arete de chaque sommet de la commu
                    poids_sommet_commu += arete[1]  #si le sommet relié au sommet en cours de traitement est aussi dans la commu on ajoute le poids de l'arete dans notre variable
                else:
                    if communautes[arete[0]] not in dico_arete_autre_commu :    #si il n'est pas dans la commu
                        dico_arete_autre_commu[communautes[arete[0]]] = arete[1] #si c'est le premier/seul elt connecté à notre commu on ajoute dans le dico la commu associé à son poids
                    else :
                        dico_arete_autre_commu[communautes[arete[0]]] += arete[1] #sinon on additionne le poids de la nouvelle arete reliant la commu
        
        for commu, poids in dico_arete_autre_commu.items():     #partie permettant de creer le nouveau graphe
            if elt not in nouveau_graphe :                      #si on n'a pas encore traité la commu
                nouveau_graphe[elt]=[(elt, poids_sommet_commu)] #on ajoute l'arete qui va sur elle meme permettant d'indiquer le poids du sommet
                nouveau_graphe[elt].append((commu, poids))      #on ajoute ensuite les aretes vers l'autre commu en cours de traitement
            else : 
                nouveau_graphe[elt].append((commu, poids))      #on a déjà la commu dans le nouveau graphe, ça veut donc dire qu'on traite les commu suivante vers laquelle il est relié

    return nouveau_graphe

In [51]:
def Louvain_Final(dico):

    nouveau_graphe = dico
    historique_graphes = [nouveau_graphe]

    while len(nouveau_graphe) > 2:
        nvelles_communautes = Louvain_P1(nouveau_graphe)
        nouveau_graphe = Louvain_P2(nouveau_graphe, nvelles_communautes)
        historique_graphes.append(nouveau_graphe)

    return historique_graphes
    

## Tests !

In [21]:
# Le graphe test contient 21 sommets et 34 arêtes

dico_test = {0:[(1,1)],
            1:[(0,1), (2,1)],
            2:[(1,1), (3,1), (4,1)],
            3:[(2,1), (4,1), (8,1)],
            4:[(2,1), (3,1), (5,1), (7,1)],
            5:[(4,1), (6,1), (17,1)],
            6:[(5,1), (7,1), (19,1)],
            7:[(4,1), (6,1), (9,1)],
            8:[(3,1)],
            9:[(7,1), (10,1), (11,1)],
            10:[(9,1), (11,1)],
            11:[(9,1), (10,1), (12,1)],
            12:[(11,1), (14,1), (17,1), (19,1)],
            13:[(14,1), (15,1), (16,1), (19,1)],
            14:[(12,1), (13,1), (15,1)],
            15:[(13,1), (14,1), (16,1), (17,1)],
            16:[(13,1), (15,1), (17,1), (18,1), (20,1)],
            17:[(5,1), (12,1), (15,1), (16,1), (18,1)],
            18:[(16,1), (17,1), (19,1), (20,1)],
            19:[(6,1), (12,1), (13,1), (18,1), (20,1)],
            20:[(16,1), (18,1), (19,1)]
            }

In [22]:
dico_test2 = {0:[(1,1), (2,1), (3,1)],
              1:[(0,1), (2,1), (3,1)],
              2:[(0,1), (1,1), (3,1)],
              3:[(0,1), (1,1), (2,1), (4,1)],
              4:[(5,1), (6,1), (7,1), (3,1)],
              5:[(4,1), (6,1), (7,1)],
              6:[(4,1), (5,1), (7,1)],
              7:[(4,1), (5,1), (6,1)],}

In [23]:
dico_SV = {0:[(2,1), (3,1), (4,1), (5,1)],
            1:[(2,1), (4,1), (7,1)],
            2:[(0,1), (1,1), (4,1), (5,1), (6,1),],
            3:[(0,1), (7,1)],
            4:[(0,1), (1,1), (2,1), (10,1)],
            5:[(0,1), (2,1), (7,1), (11,1)],
            6:[(2,1), (7,1), (11,1)],
            7:[(1,1), (3,1), (5,1), (6,1)],
            8:[(9,1), (10,1), (11,1), (14,1), (15,1)],
            9:[(8,1), (12,1), (14,1)],
            10:[(4,1), (8,1), (11,1), (12,1), (13,1), (14,1)],
            11:[(5,1), (6,1), (8,1), (10,1), (13,1)],
            12:[(9,1), (10,1)],
            13:[(10,1), (11,1)],
            14:[(8,1), (9,1), (10,1)],
            15:[(8,1)]}

In [53]:
res = Louvain_Final(dico_SV)
for d in res:
    print(d)

{0: [(2, 1), (3, 1), (4, 1), (5, 1)], 1: [(2, 1), (4, 1), (7, 1)], 2: [(0, 1), (1, 1), (4, 1), (5, 1), (6, 1)], 3: [(0, 1), (7, 1)], 4: [(0, 1), (1, 1), (2, 1), (10, 1)], 5: [(0, 1), (2, 1), (7, 1), (11, 1)], 6: [(2, 1), (7, 1), (11, 1)], 7: [(1, 1), (3, 1), (5, 1), (6, 1)], 8: [(9, 1), (10, 1), (11, 1), (14, 1), (15, 1)], 9: [(8, 1), (12, 1), (14, 1)], 10: [(4, 1), (8, 1), (11, 1), (12, 1), (13, 1), (14, 1)], 11: [(5, 1), (6, 1), (8, 1), (10, 1), (13, 1)], 12: [(9, 1), (10, 1)], 13: [(10, 1), (11, 1)], 14: [(8, 1), (9, 1), (10, 1)], 15: [(8, 1)]}
{0: [(0, 2), (1, 1), (3, 1), (2, 3)], 1: [(1, 14), (3, 4), (2, 1), (0, 1)], 2: [(2, 16), (0, 3), (1, 1)], 3: [(3, 4), (1, 4), (0, 1)]}
{0: [(0, 24), (1, 3)], 1: [(1, 26), (0, 3)]}


In [24]:
res = Louvain(dico_SV)
print(res)

noeud :  0
ki :  4
v :  2
ki_in :  1
Stot :  5
gain  0  avec  2  =  1.2857142857142856
gain max =  1.2857142857142856 avec le noeud  2
v :  3
ki_in :  1
Stot :  2
gain  0  avec  3  =  1.7142857142857144
gain max =  1.7142857142857144 avec le noeud  3
v :  4
ki_in :  1
Stot :  4
gain  0  avec  4  =  1.4285714285714286
gain max =  1.7142857142857144 avec le noeud  3
v :  5
ki_in :  1
Stot :  4
gain  0  avec  5  =  1.4285714285714286
gain max =  1.7142857142857144 avec le noeud  3
test :  0 3
je viens de modifier
communautes :  [3, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
0 3 1.7142857142857144
noeud :  1
ki :  3
v :  2
ki_in :  1
Stot :  5
gain  1  avec  2  =  1.4642857142857144
gain max =  1.4642857142857144 avec le noeud  2
v :  4
ki_in :  1
Stot :  4
gain  1  avec  4  =  1.5714285714285714
gain max =  1.5714285714285714 avec le noeud  4
v :  7
ki_in :  1
Stot :  4
gain  1  avec  7  =  1.5714285714285714
gain max =  1.5714285714285714 avec le noeud  4
test :  1 4
je viens de 

In [26]:
print(partie2(dico_SV, res))

{0: [(0, 2), (1, 1), (3, 1), (2, 3)], 1: [(1, 14), (3, 4), (2, 1), (0, 1)], 2: [(2, 16), (0, 3), (1, 1)], 3: [(3, 4), (1, 4), (0, 1)]}


In [27]:
dicoV2 = {0: [(0, 2), (1, 1), (3, 1), (2, 3)], 
          1: [(1, 14), (3, 4), (2, 1), (0, 1)], 
          2: [(2, 16), (0, 3), (1, 1)], 
          3: [(3, 4), (1, 4), (0, 1)]}

In [28]:
dicoV3 = {0: [(0,26), (1, 3)],
          1: [(1,24), (0,3)]}

In [41]:
print(partie2(dicoV2, Louvain(dicoV2)))

communautes :  [2, 1, 2, 3]
0 2 2.9565217391304346
communautes :  [2, 3, 2, 3]
1 3 4.086956521739131
communautes :  [2, 3, 2, 3]
2 0 34.95652173913044
communautes :  [2, 3, 2, 3]
3 1 12.08695652173913
communautes :  [2, 3, 2, 3]
0 2 6.956521739130435
communautes :  [2, 3, 2, 3]
1 3 32.08695652173913
communautes :  [2, 3, 2, 3]
2 0 34.95652173913044
communautes :  [2, 3, 2, 3]
3 1 12.08695652173913
{0: [(0, 24), (1, 3)], 1: [(1, 26), (0, 3)]}


In [40]:
print(Louvain(dicoV3))

communautes :  [0, 1]
0 -1 -1
communautes :  [0, 1]
1 -1 -1
[0, 1]


In [30]:
print(Louvain(partie2(dico_SV, res)))

noeud :  0
ki :  7
v :  0
v :  1
ki_in :  1
Stot :  20
gain  0  avec  1  =  -1.0434782608695654
gain max =  -1 avec le noeud  -1
v :  3
ki_in :  1
Stot :  9
gain  0  avec  3  =  0.6304347826086956
gain max =  0.6304347826086956 avec le noeud  3
v :  2
ki_in :  3
Stot :  20
gain  0  avec  2  =  2.9565217391304346
gain max =  2.9565217391304346 avec le noeud  2
test :  0 2
je viens de modifier
communautes :  [2, 1, 2, 3]
0 2 2.9565217391304346
noeud :  1
ki :  20
v :  1
v :  3
ki_in :  4
Stot :  9
gain  1  avec  3  =  4.086956521739131
gain max =  4.086956521739131 avec le noeud  3
v :  2
ki_in :  2
Stot :  27
gain  1  avec  2  =  -7.739130434782609
gain max =  4.086956521739131 avec le noeud  3
v :  0
ki_in :  2
Stot :  27
gain  1  avec  0  =  -7.739130434782609
gain max =  4.086956521739131 avec le noeud  3
test :  1 3
communautes :  [2, 1, 2, 3]
1 3 4.086956521739131
noeud :  2
ki :  20
v :  2
v :  0
ki_in :  19
Stot :  7
gain  2  avec  0  =  34.95652173913044
gain max =  34.956521739

In [31]:
def contraction_pondere(s1, s2, graphe):
    sf = f"{s1}/{s2}"
    nouveau_graphe = graphe.copy()

    for sommet in graphe:
        nouveau_graphe[sommet] = {arete for arete in graphe[sommet] if arete[0] not in {s1, s2}}
    del nouveau_graphe[s1], nouveau_graphe[s2]

In [32]:
def contraction_pondere(s1, s2, graphe):
    sf = f"{s1}/{s2}"
    nouveau_graphe = graphe.copy()

    for sommet in graphe:
        nouveau_graphe[sommet] = {arete for arete in graphe[sommet] if arete[0] not in {s1, s2}}
    del nouveau_graphe[s1], nouveau_graphe[s2]

    for sommet in nouveau_graphe:
        nouveau_graphe[sommet] = nouveau_graphe[sommet].union({(sf, sum(list(map(lambda arete: arete[1], {art for art in graphe[sommet] if art[0] in {s1, s2}}))))})
        nouveau_graphe[sommet] = {(sommet, poids) for sommet, poids in nouveau_graphe[sommet] if poids > 0}

    nouveau_graphe[sf] = set()

    for sommet in nouveau_graphe:
        nouveau_graphe[sf] = nouveau_graphe[sf].union({(sommet, arete[1]) for arete in nouveau_graphe[sommet] if arete[0] == sf}) 

    nouveau_graphe[sf] = nouveau_graphe[sf].union({(sf, sum(list(map(lambda arete: arete[1], {art for art in graphe[s1].union(graphe[s2]) if art[0] in {s1, s2}}))))})

    return nouveau_graphe

In [33]:
contraction_pondere(0,2, dico_SV)

AttributeError: 'list' object has no attribute 'union'

In [ ]:
#print(GainModularite(dico_test, communautes, m, 6, 18))

In [ ]:
commutest = [0, 1, 2, 0, 1, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
m = SommePoidsGraphe(dico_SV)
print(m)
print(GainModularite(dico_SV, commutest, m, 2, 1))

28


NameError: name 'KiInside' is not defined

In [ ]:
#print(Louvain1(dico_SV))

j'ajoute  0  à la communauté de  3
3  est maintenant dans la communauté numéro  0  et i dans la numéro  0
communautes :  [0, 1, 2, 0, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
max_gain :   -0.0076530612244897975 max_j :  4
j'ajoute  1  à la communauté de  4
4  est maintenant dans la communauté numéro  1  et i dans la numéro  1
communautes :  [0, 1, 2, 0, 1, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
i: 2 j: 0 gain: -0.01594387755102041
i: 2 j: 1 gain: -0.01913265306122449
i: 2 j: 4 gain: -0.01913265306122449
i: 2 j: 5 gain: -0.01275510204081633
i: 2 j: 6 gain: -0.009566326530612243
j'ajoute  2  à la communauté de  6
6  est maintenant dans la communauté numéro  2  et i dans la numéro  2
communautes :  [0, 1, 2, 0, 1, 5, 2, 7, 8, 9, 10, 11, 12, 13, 14, 15]
j'ajoute  3  à la communauté de  7
7  est maintenant dans la communauté numéro  0  et i dans la numéro  0
communautes :  [0, 1, 2, 0, 1, 5, 2, 0, 8, 9, 10, 11, 12, 13, 14, 15]
j'ajoute  4  à la communauté de  1
1  est maintenant dans la co

 est maintenant dans la communauté numéro  0  et i dans la numéro  0
communautes :  [0, 0, 0, 0, 0, 0, 0, 0, 0, 8, 0, 0, 8, 0, 8, 8]
j'ajoute  14  à la communauté de  9
9  est maintenant dans la communauté numéro  8  et i dans la numéro  8
communautes :  [0, 0, 0, 0, 0, 0, 0, 0, 0, 8, 0, 0, 8, 0, 8, 8]
j'ajoute  15  à la communauté de  8
8  est maintenant dans la communauté numéro  8  et i dans la numéro  8
communautes :  [0, 0, 0, 0, 0, 0, 0, 0, 8, 8, 0, 0, 8, 0, 8, 8]
j'ajoute  0  à la communauté de  2
2  est maintenant dans la communauté numéro  0  et i dans la numéro  0
communautes :  [0, 0, 0, 0, 0, 0, 0, 0, 8, 8, 0, 0, 8, 0, 8, 8]
max_gain :   0.06313775510204078 max_j :  2
j'ajoute  1  à la communauté de  2
2  est maintenant dans la communauté numéro  0  et i dans la numéro  0
communautes :  [0, 0, 0, 0, 0, 0, 0, 0, 8, 8, 0, 0, 8, 0, 8, 8]
i: 2 j: 0 gain: 0.10522959183673469
i: 2 j: 1 gain: 0.10522959183673469
i: 2 j: 4 gain: 0.10522959183673469
i: 2 j: 5 gain: 0.105229591836734

KeyboardInterrupt: 

In [ ]:
def RenvoieCommunautes(communautes):
    ''' A partir d'une liste où chaque élément d'indice i est le numéro de la communauté du noeud i,
        transforme en une autre liste ou chaque élément d'indice i est une liste des noeuds de la communauté i'''
    
    nb_commu = max(communautes)
    nv_communautes = [[] for k in range(nb_commu+1)]
    for i in range(len(communautes)):       # pour chaque noeud i
        nv_communautes[communautes[i]].append(i)

    return nv_communautes

In [ ]:
print(RenvoieCommunautes([0, 0, 0, 1, 1, 2, 2, 2, 2]))

[[0, 1, 2], [3, 4], [5, 6, 7, 8]]


In [ ]:
m = [0, 4, 6, 0, 2]
print(max(m))

6


In [ ]:
nv_communautes = [[] for k in range(5)]
print(nv_communautes)

[[], [], [], [], []]


In [ ]:
def Louvain(dico):
    
    m = SommePoidsGraphe(dico)

    # Initialisation de la liste communautes : chaque noeud est dans une communauté différente 
    communautes = [k for k in range(len(dico))]

    for noeud in communautes:
        liste_voisins = TrouveVoisins(dico, noeud)
        max = -1
        max_v = -1
        
        liste_commu_voisins = []
        for vo in liste_voisins:
            liste_commu_voisins.append(communautes[vo])
            
        ki = len(liste_voisins)
        for v in liste_voisins:
            ki_in = liste_commu_voisins.count(communautes[v])
            S_tot = 0
            n_commu_v = QuiDansCommunaute(communautes, communautes[v])      # prendre la valeur [1]
            for ncv in n_commu_v:
                if ncv != noeud :
                    S_tot += Calcul_degre(dico, ncv)

            gain = (2*ki_in) - (S_tot * ki)/m
            if gain > max:
                max = gain
                max_v = v
        
            #if noeud == 4:
                #print(v)
                #print('ki_in ', ki_in)
                #print('S-tot ', S_tot)

        AjoutDansCommunaute(communautes, noeud, communautes[max_v])

        print(noeud, max_v, max)


            